In [43]:
from collections import defaultdict
from glob import glob
import pandas as pd
import re

Throw away bad data, strip whitespace, rename stuff, etc. 

In [6]:
fftoday_files = glob("data/fftoday*.csv")
print(files)

['data/fftoday_2017_pos30.csv', 'data/fftoday_2017_pos40.csv', 'data/fftoday_2017_pos20.csv']


In [51]:
# make one big dataframe for each position
data = defaultdict(list)
for file in fftoday_files:
    df = pd.read_csv(file)
    df = df[df.columns[1:]]  # get rid of 'Unnamed: 0' column
    # group each player's entries together and put in chronological order
    # df.sort_values(['name', 'season', 'week'], inplace=True)
    # add column for next week's points?
    # normalize columns: need to have consistent column order
    # split dataframes by player? Not useful to try to get Bob's Week 12 score from Jerry's Week 12
    #data[file.strip('data/fftoday_').strip('.csv')].append(df)
    data[re.sub(r'data/fftoday_([0-9]*)_pos', '', file.strip('.csv'))].append(df)
    
for k, v in data.items():
    #print(k)
    df = pd.concat(v)
    df.sort_values(['name', 'season', 'week'], inplace=True)
    # remove players that only have 1 row, we can't use these for anything
    before =
    print(df.head(30))

                name team  g  rec_target  rec_rec  rec_yd  rec_td  rush_att  \
3         A.J. Green  CIN  1           7        5     115       1         0   
66        A.J. Green  CIN  1           9        4      50       1         0   
136       A.J. Green  CIN  1           8        5      66       0         0   
1       Adam Thielen  MIN  1          12        8     166       1         0   
54      Adam Thielen  MIN  1           9        6     123       1         0   
124     Adam Thielen  MIN  1          11        8      89       0         0   
121    Albert Wilson   KC  1           7        3      36       1         0   
67     Alex Erickson  CIN  1           2        2      42       1         0   
33       Allen Hurns  JAC  1           9        7      70       0         0   
60    Alshon Jeffery  PHI  1           7        4      67       1         0   
116   Alshon Jeffery  PHI  1           9        5      52       1         0   
72      Amari Cooper  OAK  1           7        3   